In [11]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Robert.Jones\AppData\Local\Temp\ipykernel_17204\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import format_number, mean

spark = SparkSession.builder.getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [13]:
df = spark.read.format('csv').options(inferSchema=True,Header=True).load('walmart_stock.csv')

#### What are the column names?

In [14]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [15]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [16]:
print(df.columns[0:5])

['Date', 'Open', 'High', 'Low', 'Close']


#### Use describe() to learn about the DataFrame.

In [17]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [18]:
df_describe = df.describe()

df_describe = df_describe.withColumn('Open',df_describe.Open.cast('double'))
df_describe = df_describe.withColumn('High',df_describe.High.cast('double'))
df_describe = df_describe.withColumn('Low',df_describe.Low.cast('double'))
df_describe = df_describe.withColumn('Close',df_describe.Close.cast('double'))
df_describe = df_describe.withColumn('Volume',df_describe.Volume.cast('double'))
df_describe = df_describe.withColumnRenamed('Adj Close','AdjClose')
df_describe = df_describe.withColumn('AdjClose',df_describe.AdjClose.cast('double'))

df_describe = df_describe.withColumns({'Open':format_number('Open',2),'High':format_number('High',2),'Low':format_number('Low',2),'Close':format_number('Close',2),
'Volume':format_number('Volume',2),'AdjClose':format_number('AdjCLose',2)})

df_describe.show()

+-------+--------+--------+--------+--------+-------------+--------+
|summary|    Open|    High|     Low|   Close|       Volume|AdjClose|
+-------+--------+--------+--------+--------+-------------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.48|   67.24|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,780.84|    6.72|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|   50.36|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,100.00|   84.91|
+-------+--------+--------+--------+--------+-------------+--------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [19]:
df_hv_ratio = df.withColumn('HV_Ratio',df.High / df.Volume)
df_hv_ratio.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|            HV_Ratio|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+--------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51

#### What day had the Peak High in Price?

In [20]:
df.createTempView('df')
df_peak_high = spark.sql('SELECT Date FROM df ORDER BY High DESC Limit 1')
df_peak_high.show()

+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
+-------------------+



#### What is the mean of the Close column?

In [21]:
from pyspark.sql.functions import mean

df_mean_close = df.select(mean('Close'))
df_mean_close.show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [22]:
df_max_vol = spark.sql('SELECT MAX(Volume) FROM df')
df_max_vol.show()
df_min_vol = spark.sql('SELECT MIN(Volume) FROM df')
df_min_vol.show()

+-----------+
|max(Volume)|
+-----------+
|   80898100|
+-----------+

+-----------+
|min(Volume)|
+-----------+
|    2094900|
+-----------+



#### How many days was the Close lower than 60 dollars?

In [23]:
df_lower_60 = spark.sql('SELECT Date FROM df WHERE Close < 60')
print(df_lower_60.count())

81


#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [24]:
df_higher_80 = spark.sql('SELECT Date FROM df WHERE Close > 80')
df_all = spark.sql('SELECT Date FROM df')

ratio = (df_higher_80.count() / df_all.count()) * 100
format_ratio = "{:.2f}".format(ratio)
print(format_ratio,'%')

8.74 %


#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [25]:
print(df.stat.corr('High','Volume'))

-0.3384326061737161


#### What is the max High per year?

In [26]:
df_max_per_year = spark.sql('SELECT YEAR(Date) AS year, MAX(High) AS max_high FROM df GROUP BY year ORDER BY year')
df_max_per_year.show()

+----+---------+
|year| max_high|
+----+---------+
|2012|77.599998|
|2013|81.370003|
|2014|88.089996|
|2015|90.970001|
|2016|75.190002|
+----+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [27]:
df_avg_month = spark.sql('SELECT MONTH(Date) AS month, Avg(Close) AS avg_close FROM df GROUP BY month ORDER BY month')
df_avg_month.show()

+-----+-----------------+
|month|        avg_close|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Great Job!